In [1]:
%load_ext autoreload

%autoreload 2

import pandas as pd
import numpy as np
from datetime import timezone
import datetime
import requests
import re
import json
from ast import literal_eval
import time

local_time = datetime.timedelta(hours=8)
day_delta = datetime.timedelta(days=1)
now = datetime.datetime.now(timezone.utc)
delta = datetime.datetime.now(timezone.utc) + 2*day_delta
headers = {
  'x-rapidapi-key': 'afcef025737d5fd30b78edf40de85af1',
  'x-rapidapi-host': 'v1.hockey.api-sports.io'
}
# time.sleep(6300)

In [2]:
leagues = pd.DataFrame()
season_list = ['2022', '2023']

for i in season_list:
#     for i in league_id['id'][:len_league]:
    try:
        leagues_url = "https://v1.hockey.api-sports.io/leagues?season={0}".format(i)

#             league_name = league_id[league_id['id']==i]['name'].to_list()[0]
        payload={}
        response_leagues = requests.request("GET", leagues_url, headers=headers, data=payload)

        leagues_ = response_leagues.json()
        leagues_ = pd.DataFrame(leagues_['response'])
        leagues_['season'] = i
#             round_['league'] = league_id['name'][i]
        leagues = leagues.append(leagues_)
    except:
        pass

leagues = leagues.reset_index(drop=True)
leagues_list = leagues[['id','name']]
leagues_list = pd.merge(leagues_list, pd.json_normalize(leagues['country'])['name'], left_index=True, right_index=True)
leagues_list = leagues_list.rename(columns={'name_x': 'league', 'name_y': 'country'})
leagues_list = leagues_list.sort_values('id')


In [3]:
fixtures = pd.DataFrame()
season_list = ['2022']

for j in season_list:
    for i in leagues_list['id']:
#     for i in league_id['id'][:len_league]:
        try:
            fixtures_url = "https://v1.hockey.api-sports.io/games?league={0}&season={1}".format(i,
                                                                                                     j)
            
#             league_name = league_id[league_id['id']==i]['name'].to_list()[0]
            payload2={}
            response_fixtures = requests.request("GET", fixtures_url, headers=headers, data=payload2)

            round_ = response_fixtures.json()
            round_ = pd.DataFrame(round_['response'])
            round_['season'] = j
#             round_['league'] = league_id['name'][i]
            fixtures = fixtures.append(round_)
        except:
            pass

fixtures = fixtures.reset_index(drop=True)


In [4]:
# Date filter for next matches
now = datetime.datetime.now(timezone.utc)
fixtures['id'] = fixtures['id'].apply(int)
# fixtures_filter = pd.DataFrame(fixtures['fixture'].to_list())
fixtures_filter = fixtures.copy()
fixtures_filter['date'] = fixtures_filter['date'].apply(lambda x: pd.to_datetime(x))
fixtures_filter = fixtures_filter[(fixtures_filter['date']>now) & (fixtures_filter['date']<=delta)]
fixtures_filter['date'] = fixtures_filter['date']-local_time
fixtures_filter['time'] = fixtures_filter['date'].dt.time

# Loop fixture id variable
fixture_list = fixtures_filter['id']

# List of matches with teams name

matches = pd.json_normalize(fixtures['teams'])

matches = pd.merge(matches,fixtures_filter['id'],left_index=True, right_index=True)
matches['match'] = matches['home.name']+' x '+matches['away.name']
matches = matches.rename(columns={'id': 'fixture'})
matches['sport'] = 'hockey'
# matches['fixture'] = matches['fixture'].apply(int)


In [5]:
# odds_db = pd.DataFrame()
odds_db = []

for j in season_list:
    for i in leagues_list['id']:
        try:
            odds_url = "https://v1.hockey.api-sports.io/odds?league={0}&season={1}".format(i,j)

            payload4={}
            response_odds = requests.request("GET", odds_url, headers=headers, data=payload4)

            odds = response_odds.json()
        #     odds = pd.DataFrame(odds['response'])
    #         odds['fixture'] = i
            odds_db.append(odds)
#     # ATENCAOOOO AO SLEEP
#             time.sleep(7)
        except:
            pass
    
# odds_db = odds_db.reset_index(drop=True)

In [6]:
# all_odds_loop = pd.DataFrame()

# for i in range(len(odds_db)):
#     try:
#         odds = pd.DataFrame(odds_db[i]['response'])
#         for j in range(len(odds)):
#             df = pd.json_normalize(odds['bookmakers'][j],
#                                    errors='ignore', record_prefix='_',max_level=5)

#             bookmaker_id = df['id'].to_list()
#             fixture_id = pd.json_normalize(odds['game'])['id'][j]
#             league_id = pd.json_normalize(odds['league'])['id'][j]
#             league_name = pd.json_normalize(odds['league'])['name'][j]
#             all_odds = pd.DataFrame()
            
#             for x in bookmaker_id:
#                 df_prepared = df[df['id']==x].reset_index()
#                 df_to_normalize = df_prepared['bets']
#                 bookmaker_name = df_prepared['name'][0]
#                 odd_by_bookmaker = pd.json_normalize(df_to_normalize[0],
#                                                      record_path=['values'], meta=['name'],
#                                                      errors='ignore', record_prefix='_',max_level=5)
#                 odd_by_bookmaker['bookmaker_id'] = x
#                 odd_by_bookmaker['bookmaker_name'] = bookmaker_name
#                 all_odds = all_odds.append(odd_by_bookmaker)

#             all_odds['fixture'] = fixture_id
#             all_odds['league_id'] = league_id
#             all_odds['league'] = league_name
#             all_odds_loop = all_odds_loop.append(all_odds)
#     except:
#         pass

# all_odds_loop

In [7]:
def normalize_test(odds_db):
    df_original = pd.DataFrame(pd.json_normalize(odds_db)['response']).explode('response').dropna()
    df = df_original['response'].apply(pd.json_normalize)
    df = pd.concat(df.to_list())
    step_1 = df.explode('bookmakers').reset_index(drop=True)
    step_2 = step_1.merge(pd.json_normalize(step_1['bookmakers']),
                                 left_index=True, 
                                 right_index=True).explode('bets').reset_index(drop=True)
    step_3 = step_2.merge(pd.json_normalize(step_2['bets']),
                                 left_index=True, 
                                 right_index=True).explode('values').reset_index(drop=True)
    step_4 = step_3.merge(pd.json_normalize(step_3['values']),
                                 left_index=True, 
                                 right_index=True)
    cols_new = ['_value', '_odd', 'name', 'bookmaker_id','bookmaker_name',
                'fixture', 'league_id', 'league']
    new_cols_name = {'value': '_value',
                'odd': '_odd',
                'name_y': 'name',
                'id_x': 'bookmaker_id',
                'name_x': 'bookmaker_name',
                'game.id': 'fixture',
                'league.id': 'league_id',
                'league.name': 'league'}
    step_5 = step_4.rename(columns=new_cols_name)
    step_6 = step_5[cols_new]
    
    return step_6

In [8]:
all_odds_loop = normalize_test(odds_db)

In [9]:
# League country

all_odds_loop = pd.merge(all_odds_loop, leagues_list[['id', 'country']],
                          left_on='league_id', right_on='id')
# all_odds_loop = all_odds_loop.rename(columns={'id': 'country'})
all_odds_loop['league_key'] = all_odds_loop['league']+' - '+all_odds_loop['country']
all_odds_loop

,_value,_odd,name,bookmaker_id,bookmaker_name,fixture,league_id,league,id,country,league_key
0,Home,2.25,3Way Result,1,bwin,331673,1,Extraleague,1,Belarus,Extraleague - Belarus
1,Draw,3.40,3Way Result,1,bwin,331673,1,Extraleague,1,Belarus,Extraleague - Belarus
2,Away,2.75,3Way Result,1,bwin,331673,1,Extraleague,1,Belarus,Extraleague - Belarus
3,Home,1.72,Home/Away,1,bwin,331673,1,Extraleague,1,Belarus,Extraleague - Belarus
4,Away,2.00,Home/Away,1,bwin,331673,1,Extraleague,1,Belarus,Extraleague - Belarus
...,...,...,...,...,...,...,...,...,...,...,...
512340,Away -3,1.28,Asian Handicap (Reg Time),8,1xBet,331849,132,ELJ,132,Czech-Republic,ELJ - Czech-Republic
512341,Home -2.5,2.55,Asian Handicap (Reg Time),8,1xBet,331849,132,ELJ,132,Czech-Republic,ELJ - Czech-Republic
512342,Away -2.5,1.46,Asian Handicap (Reg Time),8,1xBet,331849,132,ELJ,132,Czech-Republic,ELJ - Czech-Republic
512343,Home -3.5,3.80,Asian Handicap (Reg Time),8,1xBet,331849,132,ELJ,132,Czech-Republic,ELJ - Czech-Republic


In [10]:
# Asian Away + fix
asian_dict_fix = {}

for i in range(0,2050,50):
    if i in [0,100,200,300,400,500,600,700,800,900,1000]:
         asian_dict_fix['Away +'+str(int(i/100))] = 'Away -'+str(int(i/100))
         asian_dict_fix['Away -'+str(int(i/100))] = 'Away +'+str(int(i/100))
    else:
        asian_dict_fix['Away +'+str(i/100)] = 'Away -'+str(i/100)
        asian_dict_fix['Away -'+str(i/100)] = 'Away +'+str(i/100)

all_odds_total_final = all_odds_loop.copy()
all_odds_total_final['test'] = all_odds_total_final['_value'].map(asian_dict_fix)
all_odds_total_final['test'] = all_odds_total_final['test'].fillna(all_odds_total_final['_value'])
all_odds_total_final = all_odds_total_final.drop(columns='_value')
all_odds_total_final = all_odds_total_final.rename(columns={'test':'_value'})
cols = all_odds_total_final.columns.tolist()
cols = cols[-1:]+cols[:-1]
cols
all_odds_total_final = all_odds_total_final[cols]
# all_odds_total_final= all_odds_total_final[all_odds_total_final['bookmaker_name']!='188Bet']

In [11]:
# Asian handicap dictionary
opp_dict = {}

for i in range(0,2050,50):
    if i in [0,100,200,300,400,500,600,700,800,900,1000]:
         opp_dict['Home +'+str(int(i/100))] = 'Away -'+str(int(i/100))
         opp_dict['Away +'+str(int(i/100))] = 'Home -'+str(int(i/100))
         opp_dict['Home -'+str(int(i/100))] = 'Away +'+str(int(i/100))
         opp_dict['Away -'+str(int(i/100))] = 'Home +'+str(int(i/100))
    else:
        opp_dict['Home +'+str(i/100)] = 'Away -'+str(i/100)
        opp_dict['Away +'+str(i/100)] = 'Home -'+str(i/100)
        opp_dict['Home -'+str(i/100)] = 'Away +'+str(i/100)
        opp_dict['Away -'+str(i/100)] = 'Home +'+str(i/100)

# Over and Under dictionary
# opp_dict = {}

for i in range(0,25050,50):
    opp_dict['Over '+str(i/100)] = 'Under '+str(i/100)
    opp_dict['Under '+str(i/100)] = 'Over '+str(i/100)

# # Filter by bet type
# bet_name_list = ['Match Winner']

# all_odds_total = all_odds_loop[all_odds_loop['name'].isin(bet_name_list)]

# all_odds_total = all_odds_loop.copy()
all_odds_total_final['_odd'] = pd.to_numeric(all_odds_total_final['_odd'])

In [12]:
# Leagues filter by bookmaker

league_filter = {'bet365':
                 ['ICE Hockey League - Austria', 'Metal Ligaen - Denmark'
                  , 'Extraliga - Czech-Republic'],
                'Unibet':
                 ['DEL - Germany', 'DEL 2 - Germany', 'Extraliga - Czech-Republic',
                 'Liiga - Finland'],
                'WilliamHill':
                 ['NHL - USA'],
                'William Hill':
                 ['NHL - USA']
                ,'bwin':
                 ['ICE Hockey League - Austria', 'Elite League - United-Kingdom', 'DEL - Germany'] }
#                             , 'Extraliga']}
league_value_keys = list(league_filter.keys())

for i in league_value_keys:
    all_odds_total_final = all_odds_total_final[~(
        (all_odds_total_final['bookmaker_name']==i)&
        (all_odds_total_final['league_key'].isin(league_filter[i])))]
    
# bet filter by bookmaker

bet_filter = {'bet365':
                 ['Double Chance (1st Period)', 'Handicap Result', 'Over/Under']
             ,'Unibet': 
              ['Home Team Total Goals (2nd Period)']
             ,'WilliamHill':
              ['Asian Handicap (Reg Time)', 'Total - Home', 'Over/Under (Reg Time)'
              ,'Over/Under']
             ,'William Hill':
              ['Asian Handicap (Reg Time)', 'Total - Home', 'Over/Under (Reg Time)'
              ,'Over/Under']
             ,'bwin':
              ['Asian Handicap (Reg Time)', 'Total - Home', 'Total - Home']
             }
#                 ,'Unibet':
#                  ['DEL - Germany', 'DEL 2 - Germany', 'Extraliga - Czech-Republic',
#                  'Liiga - Finland']}
#                             , 'Extraliga']}
bet_value_keys = list(bet_filter.keys())

for i in bet_value_keys:
    all_odds_total_final = all_odds_total_final[~(
        (all_odds_total_final['bookmaker_name']==i)&
        (all_odds_total_final['name'].isin(bet_filter[i])))]
    
# bet filter by bet name & value

name_filter = {'Over/Under (Reg Time)':
                 ['Over', 'Under 4']
             ,'Total - Home': 
              ['Under 1.5']
}
#                 ,'Unibet':
#                  ['DEL - Germany', 'DEL 2 - Germany', 'Extraliga - Czech-Republic',
#                  'Liiga - Finland']}
#                             , 'Extraliga']}
name_value_keys = list(name_filter.keys())

for i in name_value_keys:
    all_odds_total_final = all_odds_total_final[~(
        (all_odds_total_final['name']==i)&
        (all_odds_total_final['_value'].str.contains('|'.join(name_filter[i]))))]

In [13]:
#list of filter bookmaker

# bookmaker_filter_ev = ['Netbet', 'SBO', '10Bet', 'Betcris']
bookmaker_filter_ev = ['1xbet', '1xBet']

all_odds_total_final = all_odds_total_final[~all_odds_total_final['bookmaker_name'].isin(bookmaker_filter_ev)]

# Unique Bookmaker per match
bookmaker_filter = ['Bet365', 'bet365', 'bwin', 'Bwin', 'Unibet', 'unibet',
                     'William Hill',  'WilliamHill',  'williamhill', 'williamhil',
                   'Pinnacle']

all_odds_total_arb = all_odds_total_final[all_odds_total_final['bookmaker_name'].isin(bookmaker_filter)]

In [14]:
best_odds = all_odds_total_arb.groupby(by=['fixture','name', '_value'])['_odd'].agg({('odd_count','count'),
                                                                                     ('_odd','max')})
best_odds = best_odds.reset_index()
best_odds['odd_perc'] = 1 / pd.to_numeric(best_odds['_odd'])

best_odds['_value_op'] = best_odds['_value'].map(opp_dict)
best_odds['_value_op'] = best_odds['_value_op'].fillna('')
best_odds['name_value'] = best_odds[['_value','_value_op']].apply(lambda x:
                                                         ' x '.join(sorted(filter(None, x), 
                                                                          reverse=True)) if 
                                                                  x['_value_op']!='' else
                                                                  x['_value_op'], axis=1)

best_odds['name_value'] = best_odds['name']+' '+best_odds['name_value']

value_odds = best_odds.groupby(by=['fixture','name_value'])[['odd_perc', 'odd_count']].sum()
value_odds = value_odds.reset_index()
value_odds = value_odds[value_odds['odd_count']>1]
value_odds = value_odds[(value_odds['odd_perc']<0.97) & (value_odds['odd_perc']>0.7)]


In [15]:
final_odds = pd.merge(best_odds, all_odds_total_arb, on=['fixture',
                                                     'name', '_value', '_odd'], how='left')
final_odds = pd.merge(final_odds, value_odds, on=['fixture','name_value'], how='inner')
final_odds = pd.merge(final_odds, matches[['match', 'fixture']], on=['fixture'], how='inner')
final_odds = final_odds[~final_odds['name'].str.contains('Winning Margin|European')]
final_odds = pd.merge(final_odds, fixtures_filter[['id','date', 'time']], left_on='fixture',
                          right_on='id', how='left')
final_odds['date_2'] = pd.to_datetime(final_odds['date']).dt.date
final_odds = final_odds.sort_values(['date_2','odd_perc_y'])
final_odds['sport'] = 'hockey'

In [16]:
## Plus EV cross bookmaker Odds

best_odds_max= all_odds_total_final.groupby(by=['fixture','name', '_value'])['_odd'].agg({('odd_count','count'),
                                                                                          ('odd_min', 'min'),
                                                                                          ('_odd','max'),
                                                                                          ('odd_mean','mean')})

best_odds_max = best_odds_max.reset_index()
best_odds_max = best_odds_max[best_odds_max['odd_count']>4]
best_odds_max['diff'] = (best_odds_max['_odd']/best_odds_max['odd_mean'])-1


best_odds_max = best_odds_max[(best_odds_max['diff']>=0.07)
                          &((best_odds_max['_odd']<=3)
                            &(best_odds_max['_odd']>=2))].sort_values('diff', ascending=False)

In [17]:
final_odds_max = pd.merge(best_odds_max, all_odds_total_final, on=['fixture',
                                                     'name', '_value', '_odd'], how='left')
# final_odds_max = pd.merge(final_odds_max, value_odds, on=['fixture','name_value'], how='inner')
final_odds_max = pd.merge(final_odds_max, matches[['match', 'fixture']], on=['fixture'], how='inner')
final_odds_max = pd.merge(final_odds_max, fixtures_filter[['id','date', 'time']],
                          left_on='fixture', right_on='id', how='inner')
# final_odds_max = pd.merge(final_odds_max, final_odds[['fixture','name', '_value', 'odd_count_y']],
#                           on=['fixture','name', '_value'], how='left')
# final_odds_max = final_odds_max[final_odds_max['odd_count_y'].isnull()]
final_odds_max['date_2'] = pd.to_datetime(final_odds_max['date']).dt.date
final_odds_max = final_odds_max.sort_values(['date_2', 'time'
                                             , 'match', 'diff'], ascending=[True, True, True, False])
final_odds_max['sport'] = 'hockey'


In [18]:
# Pinnacle strategy
odds_pinnacle = all_odds_total_final[all_odds_total_final['bookmaker_name']=='Pncl']
odds_pinnacle = odds_pinnacle[['fixture', 'name', '_value', '_odd']]
best_odds_pinnacle = pd.merge(all_odds_total_final, odds_pinnacle, on=['fixture',
                                                     'name', '_value'])
# best_odds_pinnacle[['fixture', 'name', '_value']] = best_odds_pinnacle[['fixture', 'name', '_value']].astype(str)

best_odds_pinnacle['diff'] = (best_odds_pinnacle['_odd_x']/best_odds_pinnacle['_odd_y'])-1
best_odds_pinnacle = best_odds_pinnacle[
    (best_odds_pinnacle['diff']>=0.08) 
    & (best_odds_pinnacle['_odd_x']<3) 
    & (best_odds_pinnacle['_odd_x']>=1.6)]

name_filter_pinnacle = ['Home Team Exact Goals Number' , 'Away Team Exact Goals Number']

best_odds_pinnacle = best_odds_pinnacle[~best_odds_pinnacle['name'].isin(name_filter_pinnacle)]

filter_qty_book = best_odds_pinnacle.groupby(by=['fixture','name', '_value'])['bookmaker_name'].count().reset_index()

filter_qty_book = filter_qty_book[filter_qty_book['bookmaker_name']==1]

# filter_qty_book[['fixture', 'name', '_value']] = filter_qty_book[['fixture', 'name', '_value']].astype(str)

best_odds_pinnacle = pd.merge(best_odds_pinnacle, filter_qty_book[
    ['fixture', 'name', '_value']], on=['fixture', 'name', '_value'])

best_odds_pinnacle = best_odds_pinnacle.rename(columns={'_odd_x': '_odd'})


In [19]:
final_odds_pinnacle = pd.merge(best_odds_pinnacle, matches[['match', 'fixture']], on=['fixture'], how='left')
final_odds_pinnacle = pd.merge(final_odds_pinnacle, fixtures_filter[['id','date', 'time']], left_on='fixture',
                          right_on='id', how='inner')

final_odds_pinnacle['date_2'] = pd.to_datetime(final_odds_pinnacle['date']).dt.date

final_odds_pinnacle = final_odds_pinnacle.sort_values(['date_2', 'time'
                                             , 'match', 'diff'], ascending=[True, True, True, False])
final_odds_pinnacle['sport'] = 'hockey'

cols_order_pinnacle = ['fixture', 'match', 'name', '_value', '_odd', '_odd_y'
              , 'diff', 'bookmaker_name', 'league_key', 'date_2', 'time', 'sport']

# final_odds_pinnacle['update'] = final_odds_pinnacle['update'].apply(lambda x: pd.to_datetime(x))
# final_odds_pinnacle['update'] = final_odds_pinnacle['update'] - local_time
# final_odds_pinnacle['update'] = final_odds_pinnacle['update'].dt.time

final_odds_pinnacle = final_odds_pinnacle[cols_order_pinnacle]

In [20]:
# Unique Bookmaker per match
filter_bookmaker_pinnacle = ['Bet365', 'bet365', 'Unibet', 'unibet']

final_odds_pinnacle = final_odds_pinnacle[
    final_odds_pinnacle['bookmaker_name'].isin(filter_bookmaker_pinnacle)]

# Filter by sum of diff
max_odds = final_odds_pinnacle.groupby(by=['match', 'bookmaker_name', 'name'])['diff'].max()
max_odds = max_odds.reset_index()
# max_odds = sum_odds.groupby(by=['match'])['diff'].max()
# max_odds = max_odds.reset_index()
# sum_max_odds = pd.merge(max_odds,sum_odds, on =['match', 'diff'])
# sum_max_odds = sum_max_odds.drop(columns='diff')
# max_odds[['match', 'bookmaker_name','name', 'diff']] = max_odds[['match', 'bookmaker_name','name', 'diff']].astype(str)
# final_odds_pinnacle[['match', 'bookmaker_name','name', 'diff']] = final_odds_pinnacle[['match'
#                                                                                        , 'bookmaker_name'
#                                                                                        ,'name'
#                                                                                        , 'diff']].astype(str)
#Filter final_odds
final_odds_pinnacle = pd.merge(final_odds_pinnacle
                               , max_odds[['match', 'bookmaker_name','name', 'diff']]
                               , on = ['match', 'bookmaker_name','name', 'diff'])


In [21]:
#Cleaning & order cloumns

final_odds = final_odds.drop(['_value_op','bookmaker_id','date', 'odd_count_y'], axis=1)
# final_odds_max = final_odds_max.drop(['date', 'id','bookmaker_id'], axis=1)

cols_order = ['fixture', 'match', 'name', '_value', 'odd_count', 'odd_min', '_odd', 'odd_mean'
              , 'diff', 'bookmaker_name', 'league_key', 'date_2', 'time', 'sport']

final_odds_max = final_odds_max[cols_order]

#list of filter bookmaker

bookmaker_filter = ['Netbet', 'SBO', '10Bet', 'Betcris']


final_odds_max = final_odds_max[~final_odds_max['bookmaker_name'].isin(bookmaker_filter)]

#list of filter bets

name_filter = ['Team To Score First', 'Highest Scoring Half', 'Home Team Exact Goals Number'
               , 'Away Team Exact Goals Number', 'Correct Score', 'Team To Score First'
               , 'Highest Scoring Quarter', 'To Win Either Half', 'Away Odd/Even'
               ,'Home Odd/Even', 'Odd/Even', 'Double Chance', 'Over/Under (2nd Period)'
               ,'Both Teams To Score (3rd Period)', 'Double Chance (2nd Period)'
               ,'Double Chance (1st Period)', 'Double Chance (3rd Period)']

final_odds_max = final_odds_max[~final_odds_max['name'].isin(name_filter)]

# value_filter = ['Home -2.5', 'Away -2.5']

# final_odds_max = final_odds_max[~final_odds_max['_value'].isin(name_filter)]

# team_value_filter = {'Over/Under (3rd Period)':[
#                      'Florida Panthers', 'New Jersey Devils', 'Buffalo Sabres'
#                     ,'Vancouver Canucks', 'Anaheim Ducks', 'San Jose Sharks'
#                     ,'Vegas Golden Knights', 'Toronto Maple Leafs'
#                     ,'Detroit Red Wings', 'St. Louis Blues', 'Tampa Bay Lightning']}

# name_value_keys = list(name_value_filter.keys())

# final_odds_max = final_odds_max[~((final_odds_max['name']==name_value_keys[0])&
#                                           (final_odds_max['_value'].isin(
#                                               name_value_filter[name_value_keys[0]])))]

In [22]:
# Unique Bookmaker per match
filter_bookmaker_final = ['Bet365', 'bet365', 'bwin', 'Bwin', 'Unibet', 'unibet'
                          'William Hill',  'WilliamHill',  'williamhill', 'williamhil']

final_odds_max = final_odds_max[final_odds_max['bookmaker_name'].isin(filter_bookmaker_final)]

# Filter by sum of diff
sum_odds = final_odds_max.groupby(by=['match', 'bookmaker_name'])['diff'].sum()
sum_odds = sum_odds.reset_index()
max_odds = sum_odds.groupby(by=['match'])['diff'].max()
max_odds = max_odds.reset_index()
sum_max_odds = pd.merge(max_odds,sum_odds, on =['match', 'diff'])
sum_max_odds = sum_max_odds.drop(columns='diff')

#Filter final_odds
final_odds_max = pd.merge(final_odds_max, sum_max_odds, on = ['match', 'bookmaker_name'])



In [23]:
## filter 2nd strategy

dict_name_value = [{'name': 'Over/Under (1st Period)'
                            , 'bookmaker_name':['bet365', 'Bet365'], '_value': ['Over 1.5']}
                  ,{'name': 'Asian Handicap (Reg Time)'
                            , 'bookmaker_name':'Unibet', '_value': ['Away -1', 'Home -1', 'Home -2.5']}
                  ,{'name': 'Away Team Total Goals (2nd Period)'
                            , 'bookmaker_name':'Unibet', '_value': ['Over 1.5']}
                  ,{'name': 'Over/Under (Reg Time)'
                            , 'bookmaker_name':'Unibet', '_value': ['Under 5']}
                  ,{'name': 'Total - Home'
                            , 'bookmaker_name':'Unibet', '_value': ['Over 2.5']}
                  ,{'name': 'Total - Away'
                            , 'bookmaker_name':'Unibet', '_value': ['Under 1.5']}
                  ,{'name': 'Asian Handicap'
                            , 'bookmaker_name':['bet365', 'Bet365'], '_value': ['Home -1.5']}
                  ,{'name': 'Handicap Result'
                            , 'bookmaker_name':'Unibet', '_value': ['Home -2']}
                  ,{'name': 'Home/Away'
                            , 'bookmaker_name':['bet365', 'Bet365'], '_value': ['Away']}]
#                   ,{'name': 'Over/Under (3rd Period)'
#                             , 'bookmaker_name':['bet365', 'Bet365', 'Unibet'], '_value': ['Over 2.5'
#                                                                                          ,'Over 2']}]
    

dict_name_value = pd.DataFrame.from_dict(dict_name_value).explode('bookmaker_name').explode('_value').reset_index(drop=True)

final_odds_max = pd.merge(final_odds_max, dict_name_value, on = ['name', '_value', 'bookmaker_name'])

final_odds_max = final_odds_max.sort_values(['date_2', 'time'
                                             , 'match', 'diff'], ascending=[True, True, True, False])
final_odds_max = final_odds_max.drop_duplicates()

In [54]:
fixtures_bet = pd.read_excel(r"C:\Users\Lucas\OneDrive\Documentos\Bet365\Arbitrage and + EV strategy\strategy evaluation.xlsx",
                        sheet_name='plus_ev')

fixtures_bet['date_2'] = pd.to_datetime(fixtures_bet['date_2']).dt.date

filtered_odds = pd.merge(final_odds_pinnacle, fixtures_bet,
             how='left', 
             suffixes=['', '_z'],
             on=['fixture', 'name', 'date_2'],
             indicator=True).query('_merge=="left_only"')

final_odds_pinnacle = filtered_odds.iloc[:,:12]

In [55]:
# fixtures_filter['date'] = fixtures_filter['date'].apply(lambda x: pd.to_datetime(x))

xlwriter = pd.ExcelWriter(r"C:\Users\Lucas\OneDrive\Documentos\Bet365\Arbitrage and + EV strategy\{0}"
                          .format('hockey_arb_ev.xlsx'))
# final_odds.to_excel(xlwriter, sheet_name = 'arbitrage', index=False)
# final_odds_max.to_excel(xlwriter, sheet_name = 'plus_ev', index=False)
final_odds_pinnacle.to_excel(xlwriter, sheet_name = 'pinnacle', index=False)
matches.to_excel(xlwriter, sheet_name = 'matches', index=False)
# fixtures_filter.to_excel(xlwriter, sheet_name = 'next_matches', index=False)


xlwriter.close()